<a href="https://colab.research.google.com/github/banuu0625/MA_Banu/blob/main/Chess_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle -q                         #install kaggle
! mkdir ~/.kaggle                               #make directory
! cp kaggle.json ~/.kaggle/                     #copy json file
! kaggle datasets download arevel/chess-games   #download chess games
! unzip -qq /content/chess-games.zip            #unzip file

cp: cannot stat 'kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/arevel/chess-games
License(s): CC0-1.0
100% 1.45G/1.45G [01:29<00:00, 17.9MB/s]
100% 1.45G/1.45G [01:29<00:00, 17.4MB/s]


In [ ]:
'chmod 600 /root/.kaggle/kaggle.json'

'chmod 600 /root/.kaggle/kaggle.json'

In [ ]:
! pip install chess -q #python chess package

# General Imports
import re  # For regular expressions
import gc  # For garbage collection
import os  # For file operations
import numpy as np  # Numerical computations
import pandas as pd  # For loading and handling data
import chess  # Python-chess package to manage chess moves and rules

# PyTorch Imports
import torch  # PyTorch core library
import torch.nn as nn  # Neural network layers and functions
import torch.nn.functional as F  # Non-linear activations like ReLU
from torch.utils.data import Dataset, DataLoader  # DataLoader and Dataset

# Matplotlib for visualization (optional)
import matplotlib.pyplot as plt  # For plotting results

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 12.5 MB/s eta 0:00:00


### Preprocessing the data

In [ ]:
#column index mapping - change letters into numbers and vice versa
letter_2_num = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7}
num_2_letter = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h'}

In [ ]:
#Chess board to matrix

# Idea : CNN's can acces 3D inputs !
# Create feature map for each chess piece type {pawn, knight, bishop} , white pieces =1, black pieces =-1
# CNN board can learn the rule and make moves based on the state of the game




board = chess.Board()

def board_2_rep(board):   #board object from chess package
  pieces = ['p', 'r', 'n', 'b', 'q', 'k']
  layers = []
  for piece in pieces:
    layers.append(create_rep_layer(board, piece)) #create feature map for each chess type
  board_rep = np.stack(layers) #transform feature maps into 3D-tensor
  return board_rep


def create_rep_layer(board, type):

  s = str(board)
  s = re.sub(f'[^{type}{type.upper()} \n]', '.', s) #replace everything with a '.' EXCEPT[^] desired piece (e.g pawns -> p and P)
  s = re.sub(f'{type}', '-1', s)                    #replace black pawns with -1's
  s = re.sub(f'{type.upper()}','-1',s)              #replace white pawns with 1's
  s = re.sub(f'\.', '0', s)                         #replace dots with 0's


  board_mat = []
  for row in s.split('\n'):                         #loop through lines
    row = row.split(' ')                            #split by whitespaces
    row = [int(x) for x in row]                     #replace string numbers with actual integers
    board_mat.append(row)                           #represend in np array

  return np.array(board_mat)


print(f'{board}\n')
board_2_rep(board)


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R



array([[[ 0,  0,  0,  0,  0,  0,  0,  0],
        [-1, -1, -1, -1, -1, -1, -1, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [-1, -1, -1, -1, -1, -1, -1, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  0]],

       [[-1,  0,  0,  0,  0,  0,  0, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [-1,  0,  0,  0,  0,  0,  0, -1]],

       [[ 0, -1,  0,  0,  0,  0, -1,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, -1,  0,  0,  0,  

## **Represent a move**

In [ ]:
# BASIC IDEA

#   "from matrix"             "to matrix"

# [[0 0 0 0 0 0 0 0]        [[0 0 0 0 0 0 0 0]
# [0 0 0 0 0 0 0 0]          [0 0 0 0 0 0 0 0]
# [0 0 0 0 0 0 0 0]          [0 0 0 0 0 0 0 0]
# [0 0 0 0 0 0 0 0]          [0 0 0 0 0 0 0 0]
# [0 0 0 0 0 0 0 0]          [0 0 0 0 0 1 0 0]
# [0 0 0 0 0 0 0 0]          [0 0 0 0 0 0 0 0]
# [0 0 0 0 1 0 0 0]          [0 0 0 0 0 0 0 0]
# [0 0 0 0 0 0 0 0]]         [0 0 0 0 0 0 0 0]]



def move_2_rep(move, board ):
  board.push_san(move).uci()                                    #convert dataset from SAN into UCI  format : 4 letters {d4e5}
  move = str(board.pop())

  from_output_layer = np.zeros((8,8))                           #create 2D array 8x8 zeros
  from_row = 8 - int(move[1])                                   #determine row
  from_column = letter_2_num[move[0]]                           #determine column, convert into number -> now we now the intial position(row,column)
  from_output_layer[from_row, from_column] = 1                  #convert 0 to 1 on determined position

  to_ouptut_layer = np.zeros((8,8))                             #create 2D array 8x8 zeros
  to_row = 8 - int(move[3])                                     #determine row
  to_column = letter_2_num[move[2]]                             #determine clomn -> we know the position the piece moves to
  to_ouptut_layer[to_row,to_column] = 1                         #convert 0 into a 1 on determined position

  return np.stack([from_output_layer, to_ouptut_layer])         #convert to a numpy array (stack = adding together)


move_2_rep('e2e4', board)




array([[[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]]])

## **clear SAN**

In [ ]:
def create_move_list(s):
  return re.sub('\d*\. ', '',s).split(' ')[:-1]                 #dataset = 1. (move) 2. (move), we should clear numbers followed by . (e.g 1., 2., 3.) with a white space ('')





create_move_list('1. d4 d5 2. Nf3 Nf6 3. Bf4 c6 4. e3 Bg4 5.. Be2 Nbd7')

['d4', 'd5', 'Nf3', 'Nf6', 'Bf4', 'c6', 'e3', 'Bg4', '5.Be2']

### **The Raw Data**

In [ ]:

chess_data_raw = pd.read_csv('/content/chess_games.csv', usecols =['AN','WhiteElo']) #asses only the columns we need
chess_data = chess_data_raw[chess_data_raw['WhiteElo']> 2000] #filter by elo
del chess_data_raw #delete games which are less then required elo
gc.collect() #clear RAMSPACE
chess_data = chess_data[['AN']] # del 'WhiteElo'
chess_data = chess_data[~chess_data['AN'].str.contains('{')] #clear up characters which code can't handle
chess_data = chess_data[chess_data['AN'].str.len()>20] #clear too short games
print(chess_data.shape[0])

883376


In [ ]:
print(chess_data.head(10))

                                                    AN
7    1. d4 d5 2. Nf3 Nf6 3. Bf4 c6 4. e3 Bg4 5. Be2...
8    1. d4 Nf6 2. Bf4 e6 3. e3 d5 4. Nf3 h6 5. Bd3 ...
13   1. b4 e6 2. Bb2 d5 3. Nf3 Nf6 4. b5 Be7 5. e3 ...
26   1. e4 d6 2. d4 Nf6 3. Bd3 g6 4. h4 c5 5. h5 Nx...
39   1. e4 e6 2. d4 d5 3. Nc3 Bb4 4. e5 c5 5. a3 cx...
44   1. d4 d5 2. Nc3 c6 3. Nf3 Nf6 4. Bg5 e6 5. e3 ...
75   1. Nf3 Nc6 2. d4 e5 3. e3 exd4 4. Nxd4 d5 5. N...
85   1. d4 Nf6 2. Nf3 e6 3. g3 d5 4. Bg2 c5 5. O-O ...
104  1. Nc3 d5 2. b3 d4 3. Ne4 f5 4. Ng3 f4 5. Ne4 ...
109  1. e4 Nc6 2. Nf3 d6 3. d4 Nf6 4. Nc3 g6 5. Bd3...


## **PyTorch Classes**

In [ ]:
class ChessDataset(Dataset):

  def __init__(self,games):
    super(ChessDataset, self).__init__()                      #call the constructor of its parent class
    self.games = games


  def __len__(self):
      return 40_000                                           #use 40k games, because moves are randomly selected anyway (save data?)

  def __getitem__(self, index):
    game_i = np.random.randint(self.games.shape[0])           #random integer from "Dataset" (Where did Dataset come from, what is the function of it?)
    random_game = chess_data['AN'].values[game_i]             #this integer = random game
    moves = create_move_list(random_game)                     #clean up notation by calling above created function
    game_state_i = np.random.randint(len(moves)-1)            #pick random move, and -1
    next_move = moves[game_state_i]                           #next move is game_state_i + 1
    moves = moves[:game_state_i]                              #all moves until current move
    board = chess.Board()
    for move in moves :
      board.push_san(move)                                    #loop through all moves

    x = board_2_rep(board)                                    #above function for feature map of every position
    y = move_2_rep(next_move, board)                          #feature map for played move
    if game_state_i % 2 == 1:                                 #always play positive values
      x*= -1
    return x, y




## **Dataloader**

In [ ]:
data_train = ChessDataset(chess_data['AN'])                                                 #training data with moves
data_train_loader = DataLoader(data_train, batch_size = 32, shuffle = True, drop_last=True) # 40000/32 = mini-batches with 1250 games each. shuffle for randomnes, drop last minibatch if not correct size


## **CNN**

In [ ]:
#Network = several modules one after another

class Module(nn.Module):
    def __init__(self, hidden_size):
        super(Module, self).__init__()
        self.conv1 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)          #2 convolutional layers = extract feautres, patterns
        self.conv2 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(hidden_size)                                            # batch normalisation layers = reduce overfitting and improve the generalization ability of the model
        self.bn2 = nn.BatchNorm2d(hidden_size)
        self.activation1 = nn.SELU()                                                      #activation function for to add non-linearity into the model,
        self.activation2 = nn.SELU()

    def forward(self, x):
        x_input = torch.clone(x)                                                          # Clone input for skip connection
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = x + x_input                                                                   # Residual connection, should improve learning
        x = self.activation2(x)
        return x

class ChessNet(nn.Module):
    def __init__(self, hidden_layers=4, hidden_size=200):
        super(ChessNet, self).__init__()
        self.hidden_layers = hidden_layers
        self.input_layer = nn.Conv2d(6, hidden_size, 3, stride=1, padding=1)
        self.module_list = nn.ModuleList([Module(hidden_size) for i in range(hidden_layers)]) #ModuleList connected with above "Module"
        self.output_layer = nn.Conv2d(hidden_size, 2, 3, stride=1, padding=1)

    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)

        for i in range(self.hidden_layers):
            x = self.module_list[i](x)

        x = self.output_layer(x)

        return x






## **Losses**

In [ ]:
# Assume we have already defined the neural network model as `chess_net`
model = ChessNet(hidden_layers=4, hidden_size=200).to('cuda')  # Make sure the model is on the GPU
x, y = next(iter(data_train_loader))  # Sample from DataLoader
x, y = x.float().to('cuda'), y.to('cuda')     # Move data to GPU

# Forward pass through the model
output = model(x)

# Loss calculation
metric_from = nn.CrossEntropyLoss()
metric_to = nn.CrossEntropyLoss()

# Calculating the loss for "from" and "to" positions
loss_from = metric_from(output[:,0,:], y[:,0,:])
loss_to = metric_to(output[:,1,:], y[:,1,:])
loss = loss_from + loss_to


## **Picking moves**

In [ ]:
def check_mate_single(board):
  board = board.copy()
  legal_moves = list(board.legal_moves)
  for move in legal_moves:
    board.push_uci(str(move))
    if board.is_checkmate():
      move = board.pop()
      return move
    _ = board.pop()


def distribution_over_moves(vals):
  probs = np.array(vals)
  probs = np.exp(probs)
  probs = probs/probs.sum()
  probs = probs ** 3
  probs = probs /probs.sum()
  return probs

# Predict function
def predict(x):
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # Disable gradient calculations for inference
        output = model(x)
    return output



def choose_move(board, player, color):
    legal_moves = list(board.legal_moves)

    # Check for immediate checkmate
    move = check_mate_single(board)
    if move is not None:
        return move

    # Prepare the input for the model
    x = torch.Tensor(board_2_rep(board)).float().to('cuda')
    if color == chess.BLACK:
        x *= -1
    x = x.unsqueeze(0)  # Add batch dimension
    move = predict(x)  # Model prediction, should return (1, 2, 8, 8)

    # Move output to CPU for NumPy operations
    move = move.cpu()

    vals = []
    froms = [str(legal_move)[:2] for legal_move in legal_moves]
    froms = list(set(froms))

    for from_ in froms:
        rank = 8 - int(from_[1])  # Convert rank to index (0-7)
        file = letter_2_num[from_[0]]  # Convert file (a-h) to index (0-7)

        # Ensure indexing is correct for the "from" predictions
        val = move[0, 0, rank, file].item()  # Convert tensor to scalar for compatibility with NumPy
        vals.append(val)

    # Convert values to probability distribution
    probs = distribution_over_moves(vals)

    # Choose a "from" square based on probabilities
    choosen_from = str(np.random.choice(froms, size=1, p=probs)[0])[:2]

    vals = []
    for legal_move in legal_moves:
        from_ = str(legal_move)[:2]
        if from_ == choosen_from:
            to = str(legal_move)[2:]
            rank_to = 8 - int(to[1])  # Convert rank to index
            file_to = letter_2_num[to[0]]  # Convert file to index

            # Ensure indexing is correct for the "to" predictions
            val = move[0, 1, rank_to, file_to].item()  # Convert tensor to scalar for compatibility with NumPy
            vals.append(val)
        else:
            vals.append(0)

    # Choose the final move based on the highest value in 'to' position predictions
    choosen_move = legal_moves[np.argmax(vals)]
    return choosen_move




def self_play_evaluation(board, model, color=chess.WHITE):
    # Reset the board and play a game
    board.reset()
    player = 1 if color == chess.WHITE else -1

    for move_num in range(100):  # Simulate up to 100 moves, or stop earlier if game ends
        move = choose_move(board, player, color)  # Use model to choose move
        if move is not None:
            board.push(move)  # Make the chosen move
        else:
            print(f"No legal moves available. Game over after {move_num} moves.")
            break

        if board.is_game_over():
            print(f"Game over after {move_num} moves. Result: {board.result()}")
            break

        # Switch player and color for the next move
        player *= -1
        color = chess.BLACK if color == chess.WHITE else chess.WHITE

    return board

## Train **Model**

In [ ]:
import torch.optim as optim

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # model is your neural network



num_epochs = 100
# After each epoch
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (x, y) in enumerate(data_train_loader):
        x, y = x.float().to('cuda'), y.to('cuda')

        optimizer.zero_grad()
        output = model(x)

        loss_from = metric_from(output[:, 0, :], y[:, 0])
        loss_to = metric_to(output[:, 1, :], y[:, 1])

        loss = loss_from + loss_to
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(data_train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

    # After each epoch, run self-play for evaluation
    print(f"Evaluating model after epoch {epoch + 1}")
    board = chess.Board()
    final_board = self_play_evaluation(board, model, color=chess.WHITE)
    print(final_board)

# Save the model parameters
torch.save(model.state_dict(), 'chess_net.pth')


Epoch [1/100], Step [100/1250], Loss: 0.3975
Epoch [1/100], Step [200/1250], Loss: 0.3094
Epoch [1/100], Step [300/1250], Loss: 0.2854
Epoch [1/100], Step [400/1250], Loss: 0.2822
Epoch [1/100], Step [500/1250], Loss: 0.2738
Epoch [1/100], Step [600/1250], Loss: 0.2721
Epoch [1/100], Step [700/1250], Loss: 0.2631
Epoch [1/100], Step [800/1250], Loss: 0.2652
Epoch [1/100], Step [900/1250], Loss: 0.2644
Epoch [1/100], Step [1000/1250], Loss: 0.2610
Epoch [1/100], Step [1100/1250], Loss: 0.2584
Epoch [1/100], Step [1200/1250], Loss: 0.2462
Evaluating model after epoch 1
r n b . . . . r
p p . . . p . p
. . p . p . p .
. . . p n k . .
. . . P q P . .
. . P . . N P .
P P . . B . . P
R . B Q K . . R
Epoch [2/100], Step [100/1250], Loss: 0.2479
Epoch [2/100], Step [200/1250], Loss: 0.2520
Epoch [2/100], Step [300/1250], Loss: 0.2455
Epoch [2/100], Step [400/1250], Loss: 0.2526
Epoch [2/100], Step [500/1250], Loss: 0.2486
Epoch [2/100], Step [600/1250], Loss: 0.2454
Epoch [2/100], Step [700/125

## **Evaluation**

## **UI**

In [ ]:
torch.load(model.state_dict(), 'chess_net.pth')

!pip install pygame

import pygame
from pygame.locals import *

pygame.init()


def CREATE_CHESSBOARD(window_width, window_height, surface):

    rect_width = window_width/8
    rect_height = window_height/8
    x = y = 0
    color = None
    white = (255, 255, 255)
    black = (0, 0, 0)

    def create_horizontal_rects(x, y, color, color2):
        for i in range(8):

            rects = pygame.Rect(x, y, rect_width, rect_height)

            if i % 2 == 0:
                pygame.draw.rect(surface, color, rects)
                x += rect_width

            else:
                pygame.draw.rect(surface, color2, rects)
                x += rect_width

    x = 0

    for i in range(8):

        if i % 2 == 0:
            create_horizontal_rects(x, y, white, black)
            y += rect_height

        if i % 2 == 1:
            create_horizontal_rects(x, y, black, white)
            y += rect_width


import pygame, sys


pygame.init()

WIDTH = HEIGHT = 700
WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))

def mainloop():

    while True:

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        CREATE_CHESSBOARD(WIDTH, HEIGHT, WINDOW)

        pygame.display.update()

